In [1]:
import transformers
import os
import math
print(transformers.__version__)
os.environ["CUDA_VISIBLE_DEVICES"] ="10"
from torch.functional import F

4.8.0.dev0


In [3]:
task = "pos" 
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

In [4]:
from datasets import load_dataset, load_metric
from scipy.special import softmax

In [53]:
import numpy as np
import pandas as pd
import datasets
import random
import copy
from collections import Counter,defaultdict
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer)
metric = load_metric("seqeval")
import transformers
transformers.logging.set_verbosity_error()
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

In [7]:
#!rm -r /home/ukp/.cache/huggingface/datasets/conll2003

In [8]:
data = load_dataset("conll2003")


Reusing dataset conll2003 (/home/ukp/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/40e7cb6bcc374f7c349c83acd1e9352a4f09474eb691f64f364ee62eb65d0ca6)


# Prepare data for Control Task

# Refer to Hewitt Liang et. al. (https://aclanthology.org/D19-1275.pdf)  for details about how to create control task for token classification 

In [9]:
def make_table(dt,table_prob=None,word_count=None):

    if table_prob==None: table_prob = defaultdict(lambda :defaultdict(float))
    if word_count==None: word_count = defaultdict(float)
    sentences = dt['tokens']
    tags = dt["pos_tags"]
    for sentence,tag in zip(sentences,tags):
        for i in range(len(sentence)):
            word_count[sentence[i].lower()] = word_count[sentence[i].lower()]+1
            table_prob[sentence[i].lower()][tag[i]] = table_prob[sentence[i].lower()][tag[i]] + 1
    
    return (word_count,table_prob)
    

In [10]:
word_count,table_prob = make_table(data['train'])
word_count,table_prob = make_table(data['validation'],table_prob,word_count)
word_count,table_prob = make_table(data['test'],table_prob,word_count)

In [11]:
for word in table_prob:
            for pos in table_prob[word]:
                table_prob[word][pos] = table_prob[word][pos]/word_count[word]

In [15]:
def shufs(example):
    target_tags = example['pos_tags']
    target_words = example['tokens']
    new_tags = []
    for i,j in zip(target_words,target_tags):
            new_tags.append(np.random.choice(list(table_prob[i.lower()].keys()),1,p=list(table_prob[i.lower()].values()))[0])
    if example['pos_tags']!=new_tags:
        print(example['pos_tags'],new_tags)
    example['new_pos_tag']=new_tags
    return example

In [16]:
fake_data = data.map(shufs)

Loading cached processed dataset at /home/ukp/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/40e7cb6bcc374f7c349c83acd1e9352a4f09474eb691f64f364ee62eb65d0ca6/cache-ea7bf9981e21ed64.arrow
Loading cached processed dataset at /home/ukp/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/40e7cb6bcc374f7c349c83acd1e9352a4f09474eb691f64f364ee62eb65d0ca6/cache-7f96370262389f09.arrow
Loading cached processed dataset at /home/ukp/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/40e7cb6bcc374f7c349c83acd1e9352a4f09474eb691f64f364ee62eb65d0ca6/cache-3949bcffc7cf6eef.arrow


# Control task Data Creation Done

In [18]:
from transformers import AutoTokenizer,TrainerCallback
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Preprocess Data 

In [24]:
label_all_tokens = True
def tokenize_and_align_labels(examples,tag_use=None):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[tag_use]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)
        
    
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [29]:
#tokenize_and_align_labels(fake_data['train'][:5])

In [26]:
tokenized_datasets = data.map(tokenize_and_align_labels, batched=True,remove_columns=data['train'].column_names,fn_kwargs={'tag_use':"pos_tags"})


In [28]:
tokenized_datasets_all = datasets.concatenate_datasets([tokenized_datasets['train'],tokenized_datasets['validation'],tokenized_datasets['test']])

In [31]:
#!rm -r /home/ukp/.cache/huggingface/datasets/conll2003

In [29]:
tokenized_datasets_fake = fake_data.map(tokenize_and_align_labels, batched=True,remove_columns=fake_data['train'].column_names,fn_kwargs={'tag_use':"new_pos_tag"})

In [30]:
tokenized_datasets_fake_all = datasets.concatenate_datasets([tokenized_datasets_fake['train'],tokenized_datasets_fake['validation'],tokenized_datasets_fake['test']])

In [58]:
tokenized_datasets_fake_all

Dataset({
    features: ['attention_mask', 'input_ids', 'labels'],
    num_rows: 20744
})

# Calculate Entropy for MDL and Basic evalution metrics

In [57]:
def compute_metrics(p,count=0):
    predict, labels = p
    predictions = np.argmax(predict, axis=2)
    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    
    out = softmax(predict,axis=2)
    out1=[]
    for x,y in zip(out,labels):
        temp = []
        for x1,y1 in zip(x,y):
            if y1!=-100:
                temp.append(x1[y1])  
    out1.append(-np.sum(np.log2(temp)))
    
    results = metric.compute(predictions=true_predictions, references=true_labels)
    count= count+results["overall_f1"]
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
        "count":count,
        "entropy":np.sum(out1)
    }

In [40]:
class ShowCallback(TrainerCallback):     
    def __init__(self,memory=None):
        self.memory = memory
    def on_evaluate(self, args, state, control,metrics,**kwargs):
       print(metrics)
       if state.global_step == state.max_steps:
           self.memory = self.memory + metrics['eval_entropy']
           metrics['mininum_descriptiomn_length'] = self.memory

# Break data into blocks , train on block and calculate MDL on the next successive block : This is an implementation of Voita et. al. (https://arxiv.org/pdf/2003.12298.pdf)  online coding

In [62]:
def calculate_minimum_desc_length(data_set):

    args = TrainingArguments(f"test-{task}",evaluation_strategy = "epoch",learning_rate=2e-5,
                             per_device_train_batch_size=batch_size,per_device_eval_batch_size=batch_size,
                             num_train_epochs=2,weight_decay=0.01)
    split = [1,2.5,5,10,20,30,40,60,70,100]
    data_block = [int((split[i]/100)*len(tokenized_datasets['train'])) for i in range(len(split))]
    model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))
    mdl = 14*np.log2(47)
    for i in range(1,len(data_block)-1):
        train_on = datasets.Dataset.from_dict(data_set[:data_block[i]])
        eval_on = datasets.Dataset.from_dict(data_set[data_block[i]:data_block[i+1]])
        trainer2 = Trainer(model,args,train_dataset=train_on,eval_dataset=eval_on,data_collator=data_collator,
                           tokenizer=tokenizer,compute_metrics=compute_metrics)
        trainer2.train()
        logs = trainer2.state.log_history
        mdl = mdl + [logs[i]['eval_entropy'] for i in range(len(logs)) if ('eval_entropy' in logs[i]) and logs[i]['epoch']==2.0][0]
        
    return mdl


In [63]:
calculate_minimum_desc_length(tokenized_datasets_all)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /home/ukp/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.d423bdf2f58dc8b77d5f5d18028d7ae4a72dcfd8f468e81fe979ada957a8c361
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Count,Entropy
1,No log,2.861445,0.125906,0.050271,0.071853,0.463975,0.071853,62.684032
2,No log,2.513881,0.120327,0.037251,0.056890,0.452381,0.056890,48.021004


***** Running Evaluation *****
  Num examples = 351
  Batch size = 16
***** Running Evaluation *****
  Num examples = 351
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running training *****
  Num examples = 702
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 88
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Count,Entropy
1,No log,1.636616,0.462401,0.263929,0.336049,0.549799,0.336049,83.265938
2,No log,1.404541,0.526194,0.396068,0.451951,0.623570,0.451951,66.976204


***** Running Evaluation *****
  Num examples = 702
  Batch size = 16
***** Running Evaluation *****
  Num examples = 702
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running training *****
  Num examples = 1404
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 176
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


...

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Count,Entropy
1,0.220300,0.312233,0.905209,0.909352,0.907275,0.924917,0.907275,5.702407
2,0.175700,0.320153,0.905334,0.909627,0.907476,0.924953,0.907476,7.758034


Saving model checkpoint to test-pos/checkpoint-500
Configuration saved in test-pos/checkpoint-500/config.json
Model weights saved in test-pos/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-pos/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-pos/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1404
  Batch size = 16
/home/ukp/anaconda3/envs/Sam1/lib/python3.6/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NNP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ukp/anaconda3/envs/Sam1/lib/python3.6/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ( seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ukp/anaconda3/envs/Sam1/lib/python3.6/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ) seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Count,Entropy
1,0.186800,0.255674,0.912263,0.917203,0.914726,0.934428,0.914726,6.631145
2,0.150300,0.264934,0.912979,0.917440,0.915204,0.934332,0.915204,5.673878


Saving model checkpoint to test-pos/checkpoint-500
Configuration saved in test-pos/checkpoint-500/config.json
Model weights saved in test-pos/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-pos/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-pos/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 4213
  Batch size = 16
/home/ukp/anaconda3/envs/Sam1/lib/python3.6/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NNP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ukp/anaconda3/envs/Sam1/lib/python3.6/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: VBZ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ukp/anaconda3/envs/Sam1/lib/python3.6/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: RB seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk)

248.67698200828002

In [64]:
calculate_minimum_desc_length(tokenized_datasets_fake_all)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /home/ukp/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.d423bdf2f58dc8b77d5f5d18028d7ae4a72dcfd8f468e81fe979ada957a8c361
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Count,Entropy
1,No log,2.875179,0.118659,0.049128,0.069486,0.463354,0.069486,63.519329
2,No log,2.520482,0.117034,0.035956,0.055011,0.454037,0.055011,48.556206


***** Running Evaluation *****
  Num examples = 351
  Batch size = 16
/home/ukp/anaconda3/envs/Sam1/lib/python3.6/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
***** Running Evaluation *****
  Num examples = 351
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running training *****
  Num examples = 702
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 88
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Count,Entropy
1,No log,1.665398,0.451587,0.257641,0.328096,0.542604,0.328096,106.024040
2,No log,1.437984,0.509116,0.376160,0.432654,0.612058,0.432654,91.647331


***** Running Evaluation *****
  Num examples = 702
  Batch size = 16
***** Running Evaluation *****
  Num examples = 702
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running training *****
  Num examples = 1404
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 176
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


...

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Count,Entropy
1,0.316300,0.392801,0.847215,0.845934,0.846574,0.889764,0.846574,66.995850
2,0.272600,0.394974,0.847132,0.845394,0.846262,0.890085,0.846262,69.429512


Saving model checkpoint to test-pos/checkpoint-500
Configuration saved in test-pos/checkpoint-500/config.json
Model weights saved in test-pos/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-pos/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-pos/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1404
  Batch size = 16
/home/ukp/anaconda3/envs/Sam1/lib/python3.6/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NNP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ukp/anaconda3/envs/Sam1/lib/python3.6/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ( seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ukp/anaconda3/envs/Sam1/lib/python3.6/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ) seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy,Count,Entropy
1,0.282900,0.340723,0.858223,0.852129,0.855166,0.898622,0.855166,2.068305
2,0.250800,0.346456,0.857703,0.852788,0.855238,0.898634,0.855238,1.799331


Saving model checkpoint to test-pos/checkpoint-500
Configuration saved in test-pos/checkpoint-500/config.json
Model weights saved in test-pos/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-pos/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-pos/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 4213
  Batch size = 16
/home/ukp/anaconda3/envs/Sam1/lib/python3.6/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NNP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ukp/anaconda3/envs/Sam1/lib/python3.6/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: VBZ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ukp/anaconda3/envs/Sam1/lib/python3.6/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: RB seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk)

352.97681900149746